In [1]:
import bugzilla
import requests
import time
from datetime import datetime

In [2]:
pagure_owner_alias = requests.get("https://src.fedoraproject.org/extras/pagure_owner_alias.json").json()["rpms"]

In [3]:
orphaned = [p for p in pagure_owner_alias if "orphan" in pagure_owner_alias[p]]
orphaned[:10]

['7kaa',
 '8Kingdoms',
 '915resolution',
 '99soft-oss-parent',
 'AcetoneISO2',
 'Ajaxterm',
 'CableSwig',
 'Canna',
 'CastPodder',
 'CodeAnalyst-gui']

In [4]:
bzapi = bugzilla.Bugzilla("bugzilla.redhat.com")

In [5]:
if not bzapi.logged_in:
    bzapi.interactive_login()

In [6]:
TRACKER = 1700317

query = bzapi.build_query(product="Fedora", include_fields=["flags", "assigned_to", "component", "creation_time"])
query["blocks"] = TRACKER
query["status"] = "NEW"
bugzillas = bzapi.query(query)
len(bugzillas)

274

In [7]:
RELENG = "releng@fedoraproject.org"
ORPHAN = "extras-orphan@fedoraproject.org"

In [8]:
pkgs_to_orphan = []
bugs_to_reassign = []

In [9]:
for bug in bugzillas:
    if bug.component in orphaned:
        if bug.assigned_to != ORPHAN:
            bugs_to_reassign.append(bug.id)
        continue
    needinfos = bug.get_flags("needinfo") or []
    for needinfo in needinfos:
        if needinfo['requestee'] == bug.assigned_to:
            break
    else:
        # no needinfo on assignee
        continue
    bug_state = 0
    last_time = datetime.fromtimestamp(time.mktime(bug.creation_time.timetuple()))
    for comment in bug.getcomments():
        if comment["creator"] != RELENG:
            continue
        if not comment["text"].startswith("Dear Maintainer,"):
            continue
        comment_time = datetime.fromtimestamp(time.mktime(comment["creation_time"].timetuple()))
        delta = comment_time - last_time
        if bug_state == 0 and delta.days >= 7-1:
            last_time = comment_time
            bug_state = 1
        elif bug_state == 1 and delta.days >= 3*7-1:
            last_time = comment_time
            bug_state = 2
            break
    delta = datetime.now() - last_time
    if bug_state == 2 and delta.days >= 4*7-1:
        to_orphan = f"rpms/{bug.component}"
        pkgs_to_orphan.append(to_orphan)
        print(to_orphan)
        bugs_to_reassign.append(bug.id)

rpms/nodejs-redent
rpms/eclipse-testng
rpms/cduce
rpms/eclipse-checkstyle
rpms/eclipse-m2e-mavenarchiver
rpms/nodejs-esprima
rpms/eclipse-m2e-core
rpms/fop
rpms/nodejs-finalhandler
rpms/ocaml-deriving
rpms/vdirsyncer
rpms/nodejs-make-dir
rpms/eclipse-egit
rpms/eclipse-sgx
rpms/graphite-web
rpms/lbzip2
rpms/tycho-extras
rpms/gucharmap
rpms/ocaml-sexplib
rpms/nodejs-grunt-legacy-util
rpms/nodejs-mock-git
rpms/nodejs-negotiator
rpms/rgbds
rpms/nodejs-path-type
rpms/dumb-init
rpms/golang-github-10gen-openssl
rpms/nodejs-encodeurl
rpms/eclipse-license
rpms/nodejs-jsonselect
rpms/archaius
rpms/lv2-kn0ck0ut
rpms/scamper
rpms/nodejs-temp-write
rpms/ocaml-bitstring
rpms/nodejs-mock-bin
rpms/nodejs-eyes
rpms/nodejs-load-grunt-tasks
rpms/libgovirt
rpms/nodejs-chai-fs
rpms/ocaml-json-static
rpms/erlpmd
rpms/jove
rpms/csstidy
rpms/nodejs-xmlhttprequest
rpms/zathura-cb
rpms/eclipse-anyedit
rpms/nodejs-gnode
rpms/eclipse-m2e-apt
rpms/lv2-abGate
rpms/maven-checkstyle-plugin
rpms/nodejs-moment
rpms/Exc

In [10]:
print("https://bugzilla.redhat.com/buglist.cgi?bug_id=" + ",".join(str(b) for b in bugs_to_reassign))

https://bugzilla.redhat.com/buglist.cgi?bug_id=1736290,1735164,1716525,1735008,1735116,1735139,1736200,1735135,1735216,1736205,1736340,1716546,1735855,1736243,1735122,1735161,1735310,1736046,1735984,1736988,1736929,1735327,1736358,1736222,1736270,1736276,1736588,1736281,1735104,1735282,1736192,1735130,1736237,1735458,1736083,1736644,1736302,1736474,1736337,1736249,1736203,1736241,1736000,1736177,1736346,1735193,1735890,1735057,1736321,1736995,1735112,1736218,1735133,1736080,1736095,1736272,1735414,1735257,1735147,1737002,1736096,1735803,1735162,1736173,1716488,1736217,1735800,1735186,1735052,1736499,1736308,1735250,1734998,1736228,1736195,1735132,1736292,1736176,1735058,1736149,1736294,1736335,1736353,1735142,1736174,1739634,1729908,1736186,1736989,1736279,1734993,1735115,1736478,1735187,1735148,1736637,1735270,1736579,1736180,1735118,1736189,1736540,1736188,1736190,1736050,1736208,1735127,1736282,1735117,1735267,1701227,1736215,1734940,1735113,1736465,1735125,1735145,1736191,1736056,1